In [ ]:
import os

# Define paths for train and test directories
train_dir = '/content/drive/MyDrive/Dataset/Resized/train'
test_dir = '/content/drive/MyDrive/Dataset/Resized/test'

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

print("Train and test directories created successfully.")


In [ ]:
import os
import shutil
import random

# Define paths
main_dataset_dir = '/content/drive/MyDrive/Dataset/Resized'
train_dir = '/content/drive/MyDrive/Dataset/Resized/train'
test_dir = '/content/drive/MyDrive/Dataset/Resized/test'

# Create train and test directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Define the ratio for splitting (e.g., 80% train, 20% test)
train_ratio = 0.8

# Iterate over each class directory in the main dataset directory
for class_dir in os.listdir(main_dataset_dir):
    class_path = os.path.join(main_dataset_dir, class_dir)
    if os.path.isdir(class_path):
        # Get list of image filenames in the class directory
        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

        # Shuffle the images to make sure randomness
        random.shuffle(images)

        # Calculate split indices
        split_index = int(len(images) * train_ratio)

        # Split images into train and test sets
        train_images = images[:split_index]
        test_images = images[split_index:]

        # Move images to train directory
        for image in train_images:
            src = os.path.join(class_path, image)
            dst = os.path.join(train_dir, class_dir, image)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy(src, dst)

        # Move images to test directory
        for image in test_images:
            src = os.path.join(class_path, image)
            dst = os.path.join(test_dir, class_dir, image)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy(src, dst)

print("Dataset split into train and test sets successfully.")


In [ ]:
import os

# Define the parent folder path
parent_folder = '/content/drive/MyDrive/Dataset/Resized/train'

# Function to count the number of image files in a directory
def count_images(directory):
    count = sum(1 for file in os.listdir(directory) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')))
    return count

# Iterate through each subdirectory of the parent folder
for subdir in os.listdir(parent_folder):
    subdir_path = os.path.join(parent_folder, subdir)
    if os.path.isdir(subdir_path):
        num_images = count_images(subdir_path)
        print(f"Number of images in '{subdir}': {num_images}")


In [ ]:
from PIL import Image
import os

# Specify the directory containing your images
dataset_dir = "/content/drive/MyDrive/Dataset/Resized"

# Iterate over each subfolder
for subdir, _, files in os.walk(dataset_dir):
    # Iterate over each file in the subfolder
    for file_name in files:
        # Check if the file is an image (you may need to adjust this condition depending on your file types)
        if file_name.endswith(".jpg") or file_name.endswith(".png") or file_name.endswith(".jpeg"):
            # Open the image file
            img = Image.open(os.path.join(subdir, file_name))

            # Get dimensions of the image
            width, height = img.size

            # Print dimensions
            print(f"Image: {os.path.join(subdir, file_name)}, Dimensions: {width}x{height}")


In [ ]:
import os

# Define paths for train and test directories
train_dir = '/content/drive/MyDrive/Dataset/Resized/train'
test_dir = '/content/drive/MyDrive/Dataset/Resized/test'

# Function to count the number of files in a directory
def count_files(directory):
    count = sum(len(files) for _, _, files in os.walk(directory))
    return count

# Count the number of images in train and test directories
train_image_count = count_files(train_dir)
test_image_count = count_files(test_dir)

print("Number of images in train directory:", train_image_count)
print("Number of images in test directory:", test_image_count)


In [ ]:
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from skimage import io
from sklearn.preprocessing import LabelEncoder

# Define paths for train and test directories
train_dir = '/content/drive/MyDrive/Dataset/Resized/train'
test_dir = '/content/drive/MyDrive/Dataset/Resized/test'

# Function to extract features from images in a directory
def extract_features_from_directory(directory):
    features = []
    labels = []
    for class_label in os.listdir(directory):
        class_dir = os.path.join(directory, class_label)
        if os.path.isdir(class_dir):
            for filename in os.listdir(class_dir):
                img_path = os.path.join(class_dir, filename)
                if os.path.isfile(img_path):  # Check if it's a file
                    img = io.imread(img_path)
                    features.append(img.flatten())  # Flatten image into a feature vector
                    labels.append(class_label)
                else:
                    print(f"Warning: {img_path} is not a valid file.")
    return features, labels

# Extract features and labels from training data
train_features, train_labels = extract_features_from_directory(train_dir)
test_features, test_labels = extract_features_from_directory(test_dir)

# Convert labels into numerical values
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_features, train_labels_encoded)

# Predict labels for test data
test_predictions_encoded = rf_model.predict(test_features)

# Decode predicted labels
test_predictions = label_encoder.inverse_transform(test_predictions_encoded)

# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print("Accuracy:", accuracy)


In [ ]:
import pickle

# Specify the path to save the pickle file in Google Drive
pickle_file_path = '/content/drive/MyDrive/rf_model.pkl'

# Save the trained Random Forest model to a pickle file in Google Drive
with open(pickle_file_path, 'wb') as f:
    pickle.dump(rf_model, f)

print("Pickle file saved successfully at:", pickle_file_path)


In [ ]:
import pickle
from skimage import io
import numpy as np

# Load the trained Random Forest model from the pickle file
pickle_file_path = '/content/drive/MyDrive/rf_model.pkl'
with open(pickle_file_path, 'rb') as f:
    rf_model = pickle.load(f)

# Load and preprocess the testing image
image_path = '/content/drive/MyDrive/Dataset/Resized/train/battery_resized/battery10.jpg'
test_image = io.imread(image_path)
# Preprocess the image if necessary (resize, normalize, etc.)

# Flatten the image into a feature vector
test_features = test_image.flatten().reshape(1, -1)  # Reshape to a 2D array

# Predict the class label for the testing image
predicted_label_index = rf_model.predict(test_features)[0]

# Decode the predicted label index to get the class label
# Assuming you have a label encoder to map numerical labels back to original class labels
predicted_class_label = label_encoder.inverse_transform([predicted_label_index])[0]

print("Predicted class label:", predicted_class_label)
